# Keys and Relationships Lab

## Lab Objectives
By the end of this lab, you will be able to:
- Understand different types of keys in MySQL
- Implement primary keys, unique keys, and foreign keys
- Create and manage table relationships
- Handle key constraints and referential integrity

## Prerequisites
- MySQL Server installed and running
- Python 3.x with mysql-connector-python
- Understanding of basic table creation
- Knowledge of constraints

## Lab Duration
Approximately 60 minutes

## Materials Needed
- MySQL Server
- Python environment
- This Jupyter notebook

## Key Types Overview

### Primary Key
- **Uniqueness**: Each value must be unique
- **Non-nullable**: Cannot contain NULL values
- **Single per table**: Only one primary key allowed
- **Can be composite**: Multiple columns can form a primary key

### Unique Key
- **Uniqueness**: Values must be unique (except one NULL)
- **Nullable**: Can contain NULL values
- **Multiple allowed**: Multiple unique constraints per table
- **Performance**: Can be used for fast lookups

### Foreign Key
- **References**: Points to primary key in another table
- **Referential Integrity**: Maintains relationships between tables
- **Cascade Actions**: Can define behavior on referenced data changes
- **Multiple allowed**: Can reference different tables

## Relationship Types
- **One-to-One**: One record in table A relates to one record in table B
- **One-to-Many**: One record in table A relates to many records in table B
- **Many-to-Many**: Many records in table A relate to many records in table B

## Step-by-Step Guide

First, install the required Python package:

In [ ]:
!pip install mysql-connector-python

## Step 1: Connect to MySQL

Connect to MySQL and create a practice database.

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='your_password'
)
cursor = conn.cursor()
cursor.execute('CREATE DATABASE IF NOT EXISTS keys_lab')
cursor.execute('USE keys_lab')
print('Database ready for keys and relationships practice')

## Step 2: Create Tables with Primary Keys

Create tables with different types of primary keys.

In [ ]:
# Single column primary key
cursor.execute('''
CREATE TABLE authors (
    author_id INT PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE
)
''')
print('Authors table created with single-column primary key')

In [ ]:
# Auto-increment primary key
cursor.execute('''
CREATE TABLE books (
    book_id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(200) NOT NULL,
    isbn VARCHAR(13) UNIQUE,
    publication_year INT,
    price DECIMAL(8, 2)
)
''')
print('Books table created with auto-increment primary key')

In [ ]:
# Composite primary key
cursor.execute('''
CREATE TABLE book_authors (
    book_id INT,
    author_id INT,
    contribution_type VARCHAR(20) DEFAULT 'author',
    PRIMARY KEY (book_id, author_id)
)
''')
print('Book-authors table created with composite primary key')

## Step 3: Add Foreign Key Relationships

Create foreign key constraints to establish relationships.

In [ ]:
# Add foreign keys to book_authors table
cursor.execute('''
ALTER TABLE book_authors
ADD CONSTRAINT fk_book_authors_book
FOREIGN KEY (book_id) REFERENCES books(book_id)
''')

cursor.execute('''
ALTER TABLE book_authors
ADD CONSTRAINT fk_book_authors_author
FOREIGN KEY (author_id) REFERENCES authors(author_id)
''')
print('Foreign key constraints added to book_authors table')

In [ ]:
# Create publishers table with relationship
cursor.execute('''
CREATE TABLE publishers (
    publisher_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL UNIQUE,
    address VARCHAR(255),
    phone VARCHAR(20)
)
''')

# Add publisher relationship to books
cursor.execute('''
ALTER TABLE books
ADD COLUMN publisher_id INT,
ADD CONSTRAINT fk_books_publisher
FOREIGN KEY (publisher_id) REFERENCES publishers(publisher_id)
''')
print('Publisher relationship added to books table')

## Step 4: Insert Data and Test Relationships

Insert data following the referential integrity rules.

In [ ]:
# Insert authors first (referenced table)
authors_data = [
    (1, 'George', 'Orwell', 'george.orwell@authors.com'),
    (2, 'Jane', 'Austen', 'jane.austen@authors.com'),
    (3, 'Harper', 'Lee', 'harper.lee@authors.com')
]

cursor.executemany('''
INSERT INTO authors (author_id, first_name, last_name, email)
VALUES (%s, %s, %s, %s)
''', authors_data)
conn.commit()
print(f'{cursor.rowcount} authors inserted')

In [ ]:
# Insert publishers
publishers_data = [
    ('Penguin Books', '123 Publishing St, London', '+44-20-1234-5678'),
    ('HarperCollins', '456 Book Ave, New York', '+1-212-555-1234'),
    ('Random House', '789 Literature Blvd, Toronto', '+1-416-555-5678')
]

cursor.executemany('''
INSERT INTO publishers (name, address, phone)
VALUES (%s, %s, %s)
''', publishers_data)
conn.commit()
print(f'{cursor.rowcount} publishers inserted')

In [ ]:
# Insert books (references publishers)
books_data = [
    ('1984', '9780451524935', 1949, 12.99, 1),
    ('Pride and Prejudice', '9780141439518', 1813, 9.99, 1),
    ('To Kill a Mockingbird', '9780061120084', 1960, 14.99, 2)
]

cursor.executemany('''
INSERT INTO books (title, isbn, publication_year, price, publisher_id)
VALUES (%s, %s, %s, %s, %s)
''', books_data)
conn.commit()
print(f'{cursor.rowcount} books inserted')

In [ ]:
# Insert book-author relationships
book_authors_data = [
    (1, 1, 'author'),  # 1984 by George Orwell
    (2, 2, 'author'),  # Pride and Prejudice by Jane Austen
    (3, 3, 'author')   # To Kill a Mockingbird by Harper Lee
]

cursor.executemany('''
INSERT INTO book_authors (book_id, author_id, contribution_type)
VALUES (%s, %s, %s)
''', book_authors_data)
conn.commit()
print(f'{cursor.rowcount} book-author relationships inserted')

## Step 5: Test Key Constraints

Try operations that should fail due to key constraints.

In [ ]:
# Test primary key violation
try:
    cursor.execute("INSERT INTO authors (author_id, first_name, last_name) VALUES (1, 'Duplicate', 'Author')")
    conn.commit()
    print('Primary key violation test: Unexpected success')
except mysql.connector.Error as err:
    print(f'Primary key violation test: Expected error - {err}')

In [ ]:
# Test unique key violation
try:
    cursor.execute("INSERT INTO books (title, isbn, publication_year, price) VALUES ('Duplicate ISBN', '9780451524935', 2024, 19.99)")
    conn.commit()
    print('Unique key violation test: Unexpected success')
except mysql.connector.Error as err:
    print(f'Unique key violation test: Expected error - {err}')

In [ ]:
# Test foreign key violation
try:
    cursor.execute("INSERT INTO book_authors (book_id, author_id) VALUES (999, 1)")
    conn.commit()
    print('Foreign key violation test: Unexpected success')
except mysql.connector.Error as err:
    print(f'Foreign key violation test: Expected error - {err}')

## Step 6: Query with JOINs

Use JOIN operations to query related data.

In [ ]:
# Query books with their authors
cursor.execute('''
SELECT b.title, CONCAT(a.first_name, ' ', a.last_name) as author,
       b.publication_year, b.price
FROM books b
JOIN book_authors ba ON b.book_id = ba.book_id
JOIN authors a ON ba.author_id = a.author_id
ORDER BY b.publication_year
''')

results = cursor.fetchall()
print('Books with Authors:')
print('-' * 60)
for row in results:
    print(f'{row[0]:<25} | {row[1]:<20} | {row[2]} | ${row[3]:.2f}')

In [ ]:
# Query books with publishers
cursor.execute('''
SELECT b.title, p.name as publisher, p.phone
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
ORDER BY p.name
''')

results = cursor.fetchall()
print('\nBooks with Publishers:')
print('-' * 50)
for row in results:
    print(f'{row[0]:<25} | {row[1]:<15} | {row[2]}')

## Step 7: Demonstrate CASCADE Actions

Show how foreign key actions work (conceptual demonstration).

In [ ]:
# Create a table with CASCADE DELETE to demonstrate
cursor.execute('''
CREATE TABLE reviews (
    review_id INT PRIMARY KEY AUTO_INCREMENT,
    book_id INT NOT NULL,
    reviewer_name VARCHAR(100),
    rating INT CHECK (rating >= 1 AND rating <= 5),
    review_text TEXT,
    FOREIGN KEY (book_id) REFERENCES books(book_id) ON DELETE CASCADE
)
''')

# Insert some reviews
reviews_data = [
    (1, 'Alice Johnson', 5, 'Excellent book!'),
    (1, 'Bob Smith', 4, 'Very thought-provoking'),
    (2, 'Charlie Brown', 5, 'A classic masterpiece')
]

cursor.executemany('''
INSERT INTO reviews (book_id, reviewer_name, rating, review_text)
VALUES (%s, %s, %s, %s)
''', reviews_data)
conn.commit()
print('Reviews table created with CASCADE DELETE foreign key')
print(f'{cursor.rowcount} reviews inserted')

In [ ]:
# Show CASCADE concept (don't actually delete)
cursor.execute('SELECT COUNT(*) FROM reviews')
review_count = cursor.fetchone()[0]
print(f'\nCurrent reviews count: {review_count}')
print('\nCASCADE Actions Explanation:')
print('- ON DELETE CASCADE: If a book is deleted, all its reviews are automatically deleted')
print('- ON DELETE SET NULL: If a book is deleted, foreign key is set to NULL')
print('- ON DELETE RESTRICT: Prevents deletion if related records exist')
print('- ON UPDATE CASCADE: Updates foreign key when primary key changes')

# Check current data
cursor.execute('''
SELECT b.title, COUNT(r.review_id) as review_count
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title
''')

results = cursor.fetchall()
print('\nBook Review Counts:')
for row in results:
    print(f'{row[0]}: {row[1]} reviews')

## Step 8: Clean Up

Close the database connection.

In [ ]:
cursor.close()
conn.close()
print('Database connection closed')

## Lab Summary

Excellent! You have successfully completed the Keys and Relationships Lab. In this lab, you learned how to:

1. **Implement Primary Keys**: Single-column, auto-increment, and composite primary keys
2. **Create Unique Constraints**: Ensure uniqueness for non-primary key columns
3. **Establish Foreign Keys**: Create relationships between tables with referential integrity
4. **Handle Relationship Types**: One-to-one, one-to-many, and many-to-many relationships
5. **Use JOIN Queries**: Query related data across multiple tables
6. **Understand CASCADE Actions**: Control behavior when referenced data changes

## Key Concepts Learned
- **Primary Key**: Unique, non-null identifier for each row
- **Unique Key**: Ensures uniqueness (allows one NULL)
- **Foreign Key**: Creates relationships and maintains referential integrity
- **Relationship Types**: One-to-one, one-to-many, many-to-many
- **JOIN Operations**: INNER JOIN, LEFT JOIN for querying related data
- **CASCADE Actions**: Automatic handling of related data changes

## Best Practices
- Use meaningful primary key names (id, code, etc.)
- Consider auto-increment for surrogate keys
- Use composite keys only when natural keys aren't suitable
- Always define foreign keys to maintain data integrity
- Choose appropriate CASCADE actions based on business rules
- Use JOINs efficiently to avoid performance issues

## Advanced Topics to Explore
- Database normalization (1NF, 2NF, 3NF, BCNF)
- Indexing strategies for foreign keys
- Complex many-to-many relationships
- Self-referencing foreign keys (hierarchical data)
- Database constraints vs application-level validation

## Next Steps
- Learn about advanced querying with subqueries and CTEs
- Study database design and ER modeling
- Explore performance optimization with indexes
- Build complete applications with proper database design

## Final Exercise
Design a database schema for an e-commerce system that includes:
1. **Users table** with primary key and unique email
2. **Products table** with categories (foreign key relationship)
3. **Orders table** linking users to products (many-to-many)
4. **Reviews table** for products by users
5. **Categories table** with hierarchical relationships

Include appropriate constraints and demonstrate:
- Primary key definitions
- Foreign key relationships
- Unique constraints
- JOIN queries to retrieve related data
- CASCADE actions for data integrity

Remember: Good database design starts with proper keys and relationships - they're the foundation of data integrity!